<a href="https://colab.research.google.com/github/rubenguerra/hello_world/blob/master/creacion_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q h5py pyyaml
!pip install -q tf_nightly

     |████████████████████████████████| 110.8MB 258kB/s 
     |████████████████████████████████| 512kB 45.4MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__


  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


'1.15.0-dev20190821'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
## DEFINIENDO EL MODELO
#  Se construira un modelo simple para demostrar el valor de guardar y cargar de nuevo un modelo entrenado

#Retorna un modelo de secuencia corta
def create_model():
  model = tf.keras.Sequential([
                               keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
                               keras.layers.Dropout(0.2),
                               keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model

#Se crea una instancia básica del modelo
model = create_model()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
## SE CREA UN CHECKPOINT PARA RESGUARDAR LA INFORMACIÓN EN CASO DE QUE HAYA UNA INTERRUPCIÓN DURANTE EL ENTRENAMIENTO
#  Mediante la llamada ModelCheckpoint

checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Se crea una llamada al checkpoint
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback]) # pasa la llamada al entrenamiento


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2753 - acc: 0.6334
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 324us/sample - loss: 1.1786 - acc: 0.6590 - val_loss: 0.7060 - val_acc: 0.7980
Epoch 2/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.4394 - acc: 0.8762
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 215us/sample - loss: 0.4329 - acc: 0.8760 - val_loss: 0.5284 - val_acc: 0.8460
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2810 - acc: 0.9225
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 215us/sample - loss: 0.2817 - acc: 0.9200 - val_loss: 0.4969 - val_acc: 0.8420
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2182 - acc: 0.9413
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [6]:
# Esto crea una sencilla coleccion de archivos de checkpoint que son actualizados
# al final de cada epoca

!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print('Modelo no entrenado, valor de certeza(accuracy): {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 90us/sample - loss: 2.3028 - acc: 0.1510
Modelo no entrenado, valor de certeza(accuracy): 15.10%


In [8]:
# Se cargan los valores del checkpoint, y se re-evalua

model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Modelo restaurado, certeza: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 62us/sample - loss: 0.3945 - acc: 0.8670
Modelo restaurado, certeza: 86.70%


In [10]:
##OPCION DE LLAMADA A CHECKPOINT

# Se incluye la epoca en el nombre del archivo. (Se usa 'str.format')
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Guarda los valores, cada 5 epocas
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [12]:
# Se ven los resultados y se escoge el ultimo

! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [15]:
## PARA PROBAR, REINICIAR EL MODELO Y CARGAR EL ÚLTIMO CHECKPOINT

model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print('Modelo restaurado, certeza (accuracy): {:5.2f}'.format(100*acc))

1000/1000 [==============================] - 0s 87us/sample - loss: 0.4777 - acc: 0.8840
Modelo restaurado, certeza (accuracy): 88.40


In [19]:
## PARA GUARDAR LOS VALORES MANUALMENTE
#  Se usa el metodo Model.save_weights

# Salvar valores
model.save_weights('./checkpoints/my_checkpoint')

#Restaurar los valores
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print('Modelo restaurado, certeza (accuracy): {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 106us/sample - loss: 2.2844 - acc: 0.1410
Modelo restaurado, certeza (accuracy): 14.10


In [20]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

#Salva el modelo completo en un archivo HDF5
model.save('mi_modelo.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 200us/sample - loss: 1.1675 - acc: 0.6500
Epoch 2/5
1000/1000 [==============================] - 0s 136us/sample - loss: 0.4218 - acc: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 145us/sample - loss: 0.2879 - acc: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 147us/sample - loss: 0.2113 - acc: 0.9470
Epoch 5/5
1000/1000 [==============================] - 0s 140us/sample - loss: 0.1556 - acc: 0.9660


In [22]:
# Ahora, de este modo se recrea el modelo exacto desde el archivo
# incluyendo los valores y el optimizador

new_model = keras.models.load_model('mi_modelo.h5')
new_model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [23]:
## Se chequea la certeza

loss, acc = new_model.evaluate(test_images, test_labels)
print('Modelo restaurado, certeza (accuracy): {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 112us/sample - loss: 0.4235 - acc: 0.8660
Modelo restaurado, certeza (accuracy): 86.60%


In [24]:
## SE CREA UN NUEVO MODELO

model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 208us/sample - loss: 1.1627 - acc: 0.6590
Epoch 2/5
1000/1000 [==============================] - 0s 137us/sample - loss: 0.4294 - acc: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 139us/sample - loss: 0.2907 - acc: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 141us/sample - loss: 0.2128 - acc: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 144us/sample - loss: 0.1640 - acc: 0.9630


In [26]:
# CREAMOS UN MODELO_GUARDADO (saved_model)

import time

saved_model_path = './saved_models/' + str(int(time.time()))
tf.contrib.saved_model.save_keras_model(model, saved_model_path)


Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatu

In [27]:
# Echemos un vistazo
!ls {saved_model_path}

assets	saved_model.pb	variables


In [28]:
# Recarga un nuevo modelo desde el modelo guardado

new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
## CORRIENDO EL NUEVO MODELO

# Debe ser compilado antes de evaluarlo

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

#Evaluamos el modelo restaurado
loss, acc = new_model.evaluate(test_images, test_labels)
print('Modelo restaurado, certeza: {:5.2f}%'.format(acc*100))

1000/1000 [==============================] - 0s 117us/sample - loss: 0.4313 - acc: 0.8590
Modelo restaurado, certeza: 85.90%
